In [ ]:
from narrative_llm_agent.crews.job_crew import JobCrew
from langchain_openai import ChatOpenAI, OpenAI
import openai
import os
from crewai import Crew, Agent, Task
from narrative_llm_agent.agents.kbase_agent import KBaseAgent
from langchain.tools import tool
import json
from pydantic import BaseModel
from narrative_llm_agent.agents.metadata import MetadataAgent
from narrative_llm_agent.agents.analyst import AnalystAgent
import re

In [2]:
import os
import requests


response = requests.get(
   "https://api.cborg.lbl.gov/key/info",
   headers={"Authorization": "Bearer " + os.environ.get("CBORG_API_KEY")})


print("Current spend: ", response.json()["info"]["spend"])

Current spend:  1.45803


In [3]:
# Test out the split run with first go to the validaor and then to the workflow runner

steps_to_run =  [
    {
      "Step": 1,
      "Name": "Assess Read Quality",
      "App": "Assess Read Quality with FastQC",
      "Description": "Evaluate the quality of raw reads using FastQC.",
      "expect_new_object": False,
      "app_id": "kb_fastqc/runFastQC",
      "input_data_object": ["paired_end_reads"],
      "output_data_object": ["fastqc_report"]
    },
    {
      "Step": 2,
      "Name": "Trim Reads",
      "App": "Trim Reads with Trimmomatic",
      "Description": "Trim adapter sequences and low-quality bases from reads using Trimmomatic.",
      "expect_new_object": True,
      "app_id": "kb_trimmomatic/run_trimmomatic",
      "input_data_object": ["paired_end_reads"],
      "output_data_object": ["trimmed_reads"]
    },
    {
      "Step": 3,
      "Name": "Assemble Reads",
      "App": "Assemble Reads with SPAdes",
      "Description": "Assemble trimmed reads into a draft genome assembly using SPAdes.",
      "expect_new_object": True,
      "app_id": "kb_SPAdes/run_SPAdes",
      "input_data_object": ["trimmed_reads"],
      "output_data_object": ["genome_assembly"]
    },
    {
      "Step": 4,
      "Name": "Assess Assembly Quality",
      "App": "Assess Quality of Assemblies with QUAST",
      "Description": "Evaluate assembly metrics (e.g., contig count, N50) using QUAST.",
      "expect_new_object": False,
      "app_id": "kb_quast/run_QUAST_app",
      "input_data_object": ["genome_assembly"],
      "output_data_object": ["quast_report"]
    },
    {
      "Step": 5,
      "Name": "Check Genome Completeness",
      "App": "Assess Genome Quality with CheckM",
      "Description": "Assess completeness and contamination of the assembly using CheckM.",
      "expect_new_object": False,
      "app_id": "kb_Msuite/run_checkM_lineage_wf",
      "input_data_object": ["genome_assembly"],
      "output_data_object": ["checkM_report"]
    },
    {
      "Step": 6,
      "Name": "Annotate Genome",
      "App": "Annotate Metagenome Assembly with Prokka",
      "Description": "Annotate the assembly using Prokka.",
      "expect_new_object": True,
      "app_id": "ProkkaAnnotation/annotate_metagenome",
      "input_data_object": ["genome_assembly"],
      "output_data_object": ["annotated_genome"]
    },
    {
      "Step": 7,
      "Name": "Classify Microbe Taxonomically",
      "App": "Classify Microbes with GTDB-Tk",
      "Description": "Obtain taxonomic classification of the annotated genome using GTDB-Tk.",
      "expect_new_object": False,
      "app_id": "kb_gtdbtk/run_kb_gtdbtk_classify_wf",
      "input_data_object": ["annotated_genome"],
      "output_data_object": ["gtdbtk_classification_report"]
    }
  ]


In [4]:
steps_to_run

[{'Step': 1,
  'Name': 'Assess Read Quality',
  'App': 'Assess Read Quality with FastQC',
  'Description': 'Evaluate the quality of raw reads using FastQC.',
  'expect_new_object': False,
  'app_id': 'kb_fastqc/runFastQC',
  'input_data_object': ['paired_end_reads'],
  'output_data_object': ['fastqc_report']},
 {'Step': 2,
  'Name': 'Trim Reads',
  'App': 'Trim Reads with Trimmomatic',
  'Description': 'Trim adapter sequences and low-quality bases from reads using Trimmomatic.',
  'expect_new_object': True,
  'app_id': 'kb_trimmomatic/run_trimmomatic',
  'input_data_object': ['paired_end_reads'],
  'output_data_object': ['trimmed_reads']},
 {'Step': 3,
  'Name': 'Assemble Reads',
  'App': 'Assemble Reads with SPAdes',
  'Description': 'Assemble trimmed reads into a draft genome assembly using SPAdes.',
  'expect_new_object': True,
  'app_id': 'kb_SPAdes/run_SPAdes',
  'input_data_object': ['trimmed_reads'],
  'output_data_object': ['genome_assembly']},
 {'Step': 4,
  'Name': 'Assess As

In [5]:
from narrative_llm_agent.workflow_graph.nodes import WorkflowState, WorkflowNodes
from typing import List, Dict, Any, Optional, TypedDict
narrative_id = "215269"
reads_id = "215269/2/1"
class WorkflowState(TypedDict):
    description: str 
    steps_to_run: List[Dict[str, Any]]
    last_executed_step: Dict[str, Any]
    narrative_id: int
    reads_id: str
    step_result: Optional[str]
    error: Optional[str]
    results: Optional[str]
# Initialize the state
initial_state = {
    "narrative_id": narrative_id,
    "reads_id": reads_id,
    "description": "something",
    "analysis_plan": steps_to_run,
    "steps_to_run": steps_to_run,
    "results": None,
    "error": None,
    "last_executed_step": {}  # Initialize with empty dict to prevent KeyError
}
nodes = WorkflowNodes()
state_n1 = nodes.workflow_validator_node(initial_state)

# Agent: You are a workflow validator, responsible for analyzing app run results and determining next steps.
## Task: 
                Analyze the result of the last executed step and determine if the next planned step is appropriate.
                
                Last step executed:
                {}
                
                Result of the last step:
                
                
                
                Next planned step:
                {"Step": 1, "Name": "Assess Read Quality", "App": "Assess Read Quality with FastQC", "Description": "Evaluate the quality of raw reads using FastQC.", "expect_new_object": false, "app_id": "kb_fastqc/runFastQC", "input_data_object": ["paired_end_reads"], "output_data_object": ["fastqc_report"]}
                If this is the first step, i.e. Last step executed is None, then the input object for this step should be paired-end reads object with id 215269/2/1.
                Based on the outcome of the last step, evaluate if the 

In [6]:
state_n1

{'narrative_id': '215269',
 'reads_id': '215269/2/1',
 'description': 'something',
 'analysis_plan': [{'Step': 1,
   'Name': 'Assess Read Quality',
   'App': 'Assess Read Quality with FastQC',
   'Description': 'Evaluate the quality of raw reads using FastQC.',
   'expect_new_object': False,
   'app_id': 'kb_fastqc/runFastQC',
   'input_data_object': ['paired_end_reads'],
   'output_data_object': ['fastqc_report']},
  {'Step': 2,
   'Name': 'Trim Reads',
   'App': 'Trim Reads with Trimmomatic',
   'Description': 'Trim adapter sequences and low-quality bases from reads using Trimmomatic.',
   'expect_new_object': True,
   'app_id': 'kb_trimmomatic/run_trimmomatic',
   'input_data_object': ['paired_end_reads'],
   'output_data_object': ['trimmed_reads']},
  {'Step': 3,
   'Name': 'Assemble Reads',
   'App': 'Assemble Reads with SPAdes',
   'Description': 'Assemble trimmed reads into a draft genome assembly using SPAdes.',
   'expect_new_object': True,
   'app_id': 'kb_SPAdes/run_SPAdes',

In [15]:
import re

def extract_upa(text):
    # Pattern to match UPA format: numbers/numbers/numbers
    # Can be adjusted if the format is different
    upa_pattern = r'\d+/\d+/\d+'
    
    # Search for the pattern in the text
    match = re.search(upa_pattern, text)
    
    # Return the found UPA or None if not found
    return match.group(0) if match else None

# Example usage
validation_text = 'There is no last step to evaluate, so we will proceed with the planned first step using the paired-end reads object (215269/2/1) as input for FastQC.'
upa = extract_upa(state_n1["validation_reasoning"])
print(f"Extracted UPA: {upa}")

Extracted UPA: 215269/2/1


In [7]:
state_n2 = nodes.workflow_runner_node(state_n1)

Overriding of current TracerProvider is not allowed


current step to run: {'Step': 1, 'Name': 'Assess Read Quality', 'App': 'Assess Read Quality with FastQC', 'Description': 'Evaluate the quality of raw reads using FastQC.', 'expect_new_object': False, 'app_id': 'kb_fastqc/runFastQC', 'input_data_object': ['paired_end_reads'], 'output_data_object': ['fastqc_report']}
Running execution crew...
# Agent: KBase workflow runner
## Task: 
                This task involves running an app, this app is a part of a workflow where the output of one app (if any) is fed into the next as input. 
                Here is the current the task in JSON format: {"Step": 1, "Name": "Assess Read Quality", "App": "Assess Read Quality with FastQC", "Description": "Evaluate the quality of raw reads using FastQC.", "expect_new_object": false, "app_id": "kb_fastqc/runFastQC", "input_data_object": ["paired_end_reads"], "output_data_object": ["fastqc_report"]}.
                If any task has "expect_new_object" set to True, then that should receive a new data obje

Overriding of current TracerProvider is not allowed


starting an app run: 215269 kb_fastqc/runFastQC 215269/2/1
# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_fastqc/runFastQC, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "215269/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object.
            The new object name should be based on the input object name, not its UPA. But it must NEVER be identical to the input object name,
            always create a new name.
            If

In [8]:
state_n2

{'narrative_id': '215269',
 'reads_id': '215269/2/1',
 'description': 'something',
 'analysis_plan': [{'Step': 1,
   'Name': 'Assess Read Quality',
   'App': 'Assess Read Quality with FastQC',
   'Description': 'Evaluate the quality of raw reads using FastQC.',
   'expect_new_object': False,
   'app_id': 'kb_fastqc/runFastQC',
   'input_data_object': ['paired_end_reads'],
   'output_data_object': ['fastqc_report']},
  {'Step': 2,
   'Name': 'Trim Reads',
   'App': 'Trim Reads with Trimmomatic',
   'Description': 'Trim adapter sequences and low-quality bases from reads using Trimmomatic.',
   'expect_new_object': True,
   'app_id': 'kb_trimmomatic/run_trimmomatic',
   'input_data_object': ['paired_end_reads'],
   'output_data_object': ['trimmed_reads']},
  {'Step': 3,
   'Name': 'Assemble Reads',
   'App': 'Assemble Reads with SPAdes',
   'Description': 'Assemble trimmed reads into a draft genome assembly using SPAdes.',
   'expect_new_object': True,
   'app_id': 'kb_SPAdes/run_SPAdes',

In [9]:
state_n3 = nodes.workflow_validator_node(state_n2)

Overriding of current TracerProvider is not allowed


# Agent: You are a workflow validator, responsible for analyzing app run results and determining next steps.
## Task: 
                Analyze the result of the last executed step and determine if the next planned step is appropriate.
                
                Last step executed:
                {"Step": 1, "Name": "Assess Read Quality", "App": "Assess Read Quality with FastQC", "Description": "Evaluate the quality of raw reads using FastQC.", "expect_new_object": false, "app_id": "kb_fastqc/runFastQC", "input_data_object": ["paired_end_reads"], "output_data_object": ["fastqc_report"]}
                
                Result of the last step:
                - The "Assess Read Quality with FastQC" app was successfully run using input object UPA "215269/2/1" in narrative ID 215269.
- No new data object was created, as expected.
- The report generated by FastQC is stored at UPA "215269/3/1".
- 1 app was run in total.
```
                
                
                Next plann

In [10]:
state_n4 = nodes.workflow_runner_node(state_n3)

Overriding of current TracerProvider is not allowed


current step to run: {'Step': 2, 'Name': 'Trim Reads', 'App': 'Trim Reads with Trimmomatic', 'Description': 'Trim adapter sequences and low-quality bases from reads using Trimmomatic.', 'expect_new_object': True, 'app_id': 'kb_trimmomatic/run_trimmomatic', 'input_data_object': ['paired_end_reads'], 'output_data_object': ['trimmed_reads']}
Running execution crew...
# Agent: KBase workflow runner
## Task: 
                This task involves running an app, this app is a part of a workflow where the output of one app (if any) is fed into the next as input. 
                Here is the current the task in JSON format: {"Step": 2, "Name": "Trim Reads", "App": "Trim Reads with Trimmomatic", "Description": "Trim adapter sequences and low-quality bases from reads using Trimmomatic.", "expect_new_object": true, "app_id": "kb_trimmomatic/run_trimmomatic", "input_data_object": ["paired_end_reads"], "output_data_object": ["trimmed_reads"]}.
                If any task has "expect_new_object" set t

Overriding of current TracerProvider is not allowed


starting an app run: 215269 kb_trimmomatic/run_trimmomatic 215269/2/1
# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_trimmomatic/run_trimmomatic, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "215269/2/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object.
            The new object name should be based on the input object name, not its UPA. But it must NEVER be identical to the input object name,
            always create a ne

In [11]:
state_n5 = nodes.workflow_validator_node(state_n4)

Overriding of current TracerProvider is not allowed


# Agent: You are a workflow validator, responsible for analyzing app run results and determining next steps.
## Task: 
                Analyze the result of the last executed step and determine if the next planned step is appropriate.
                
                Last step executed:
                {"Step": 2, "Name": "Trim Reads", "App": "Trim Reads with Trimmomatic", "Description": "Trim adapter sequences and low-quality bases from reads using Trimmomatic.", "expect_new_object": true, "app_id": "kb_trimmomatic/run_trimmomatic", "input_data_object": ["paired_end_reads"], "output_data_object": ["trimmed_reads"]}
                
                Result of the last step:
                The "Trim Reads with Trimmomatic" app was successfully run in narrative 215269 using the input object UPA "215269/2/1". The step produced new output objects: 
1. Bsubtilis_rawdata_trimmed_paired with UPA "215269/4/1"
2. Bsubtilis_rawdata_trimmed_unpaired_fwd with UPA "215269/5/1"
3. Bsubtilis_rawdata_

In [12]:
state_n6 = nodes.workflow_runner_node(state_n5)

Overriding of current TracerProvider is not allowed


current step to run: {'Step': 3, 'Name': 'Assemble Reads', 'App': 'Assemble Reads with SPAdes', 'Description': 'Assemble trimmed reads into a draft genome assembly using SPAdes.', 'expect_new_object': True, 'app_id': 'kb_SPAdes/run_SPAdes', 'input_data_object': ['trimmed_reads'], 'output_data_object': ['genome_assembly']}
Running execution crew...
# Agent: KBase workflow runner
## Task: 
                This task involves running an app, this app is a part of a workflow where the output of one app (if any) is fed into the next as input. 
                Here is the current the task in JSON format: {"Step": 3, "Name": "Assemble Reads", "App": "Assemble Reads with SPAdes", "Description": "Assemble trimmed reads into a draft genome assembly using SPAdes.", "expect_new_object": true, "app_id": "kb_SPAdes/run_SPAdes", "input_data_object": ["trimmed_reads"], "output_data_object": ["genome_assembly"]}.
                If any task has "expect_new_object" set to True, then that should receive a

Overriding of current TracerProvider is not allowed


starting an app run: 215269 kb_SPAdes/run_SPAdes 215269/4/1
# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_SPAdes/run_SPAdes, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "215269/4/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object.
            The new object name should be based on the input object name, not its UPA. But it must NEVER be identical to the input object name,
            always create a new name.
            

In [13]:
state_n7 = nodes.workflow_validator_node(state_n6)

Overriding of current TracerProvider is not allowed


# Agent: You are a workflow validator, responsible for analyzing app run results and determining next steps.
## Task: 
                Analyze the result of the last executed step and determine if the next planned step is appropriate.
                
                Last step executed:
                {"Step": 3, "Name": "Assemble Reads", "App": "Assemble Reads with SPAdes", "Description": "Assemble trimmed reads into a draft genome assembly using SPAdes.", "expect_new_object": true, "app_id": "kb_SPAdes/run_SPAdes", "input_data_object": ["trimmed_reads"], "output_data_object": ["genome_assembly"]}
                
                Result of the last step:
                - App Run: kb_SPAdes/run_SPAdes
- Narrative ID: 215269
- Input Object UPA: 215269/4/1 (trimmed_reads)
- Output Object: Created genome assembly object with UPA 215269/8/1
- Number of Apps Run: 1
```
                
                
                Next planned step:
                {"Step": 4, "Name": "Assess Assembly

In [14]:
state_n8 = nodes.workflow_validator_node(state_n7)

Overriding of current TracerProvider is not allowed


# Agent: You are a workflow validator, responsible for analyzing app run results and determining next steps.
## Task: 
                Analyze the result of the last executed step and determine if the next planned step is appropriate.
                
                Last step executed:
                {"Step": 3, "Name": "Assemble Reads", "App": "Assemble Reads with SPAdes", "Description": "Assemble trimmed reads into a draft genome assembly using SPAdes.", "expect_new_object": true, "app_id": "kb_SPAdes/run_SPAdes", "input_data_object": ["trimmed_reads"], "output_data_object": ["genome_assembly"]}
                
                Result of the last step:
                - App Run: kb_SPAdes/run_SPAdes
- Narrative ID: 215269
- Input Object UPA: 215269/4/1 (trimmed_reads)
- Output Object: Created genome assembly object with UPA 215269/8/1
- Number of Apps Run: 1
```
                
                
                Next planned step:
                {"Step": 4, "Name": "Assess Assembly

In [16]:
state_n9 = nodes.workflow_runner_node(state_n8)

Overriding of current TracerProvider is not allowed


current step to run: {'Step': 4, 'Name': 'Assess Assembly Quality', 'App': 'Assess Quality of Assemblies with QUAST', 'Description': 'Evaluate assembly metrics (e.g., contig count, N50) using QUAST.', 'expect_new_object': False, 'app_id': 'kb_quast/run_QUAST_app', 'input_data_object': ['genome_assembly'], 'output_data_object': ['quast_report']}
Running execution crew...
# Agent: KBase workflow runner
## Task: 
                This task involves running an app, this app is a part of a workflow where the output of one app (if any) is fed into the next as input. 
                Here is the current the task in JSON format: {"Step": 4, "Name": "Assess Assembly Quality", "App": "Assess Quality of Assemblies with QUAST", "Description": "Evaluate assembly metrics (e.g., contig count, N50) using QUAST.", "expect_new_object": false, "app_id": "kb_quast/run_QUAST_app", "input_data_object": ["genome_assembly"], "output_data_object": ["quast_report"]}.
                If any task has "expect_new_o

Overriding of current TracerProvider is not allowed


starting an app run: 215269 kb_quast/run_QUAST_app 215269/8/1
# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_quast/run_QUAST_app, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "215269/8/1", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object.
            The new object name should be based on the input object name, not its UPA. But it must NEVER be identical to the input object name,
            always create a new name.
        

In [17]:
state_n10 = nodes.workflow_validator_node(state_n9)

Overriding of current TracerProvider is not allowed


# Agent: You are a workflow validator, responsible for analyzing app run results and determining next steps.
## Task: 
                Analyze the result of the last executed step and determine if the next planned step is appropriate.
                
                Last step executed:
                {"Step": 4, "Name": "Assess Assembly Quality", "App": "Assess Quality of Assemblies with QUAST", "Description": "Evaluate assembly metrics (e.g., contig count, N50) using QUAST.", "expect_new_object": false, "app_id": "kb_quast/run_QUAST_app", "input_data_object": ["genome_assembly"], "output_data_object": ["quast_report"]}
                
                Result of the last step:
                The task was successfully completed by running one app: "Assess Quality of Assemblies with QUAST". The task did not produce any new UPA since "expect_new_object" was false, but it generated a report upa '215269/10/1' as output. The task was executed in the narrative 215269.
```
                


In [18]:
state_n11 = nodes.workflow_runner_node(state_n10)

Overriding of current TracerProvider is not allowed


current step to run: {'Step': 5, 'Name': 'Check Genome Completeness', 'App': 'Assess Genome Quality with CheckM', 'Description': 'Assess completeness and contamination of the assembly using CheckM.', 'expect_new_object': False, 'app_id': 'kb_Msuite/run_checkM_lineage_wf', 'input_data_object': ['genome_assembly'], 'output_data_object': ['checkM_report']}
Running execution crew...
# Agent: KBase workflow runner
## Task: 
                This task involves running an app, this app is a part of a workflow where the output of one app (if any) is fed into the next as input. 
                Here is the current the task in JSON format: {"Step": 5, "Name": "Check Genome Completeness", "App": "Assess Genome Quality with CheckM", "Description": "Assess completeness and contamination of the assembly using CheckM.", "expect_new_object": false, "app_id": "kb_Msuite/run_checkM_lineage_wf", "input_data_object": ["genome_assembly"], "output_data_object": ["checkM_report"]}.
                If any task

Overriding of current TracerProvider is not allowed


starting an app run: 215269 kb_Msuite/run_checkM_lineage_wf genome_assembly
# Agent: Project coordinator
## Task: 
            From the given KBase app id, kb_Msuite/run_checkM_lineage_wf, fetch the list of parameters needed to run it. Use the App and Job manager agent
            for assistance. Using the data object with UPA "genome_assembly", populate a dictionary
            with the parameters where the keys are parameter ids, and values are the proper parameter values, or their
            default values if no value can be found or calculated.
            Any input object parameter must be the input object UPA.
            Be sure to make sure there is a non-null value for any parameter that is not optional.
            Any parameter that has a true value for "is_output_object" must have a valid name for the new object.
            The new object name should be based on the input object name, not its UPA. But it must NEVER be identical to the input object name,
            always

In [19]:
state_n12 = nodes.workflow_validator_node(state_n11)

Overriding of current TracerProvider is not allowed


# Agent: You are a workflow validator, responsible for analyzing app run results and determining next steps.
## Task: 
                Analyze the result of the last executed step and determine if the next planned step is appropriate.
                
                Last step executed:
                {"Step": 5, "Name": "Check Genome Completeness", "App": "Assess Genome Quality with CheckM", "Description": "Assess completeness and contamination of the assembly using CheckM.", "expect_new_object": false, "app_id": "kb_Msuite/run_checkM_lineage_wf", "input_data_object": ["genome_assembly"], "output_data_object": ["checkM_report"]}
                
                Result of the last step:
                The workflow was halted due to an Error in Step 5 "Check Genome Completeness" using the app "Assess Genome Quality with CheckM". The application could not run because the input object UPA "genome_assembly" was incorrect. Therefore, no additional steps were completed, and no new output o